In [1]:
import sys
import os
import torch
import networkx as nx
import pandas as pd

src_path = os.path.abspath(os.path.join('..', 'src'))
sys.path.append(src_path)

from transformers import pipeline


src_path = os.path.abspath(os.path.join('..', 'src'))
sys.path.append(src_path)

In [ ]:
file_path = r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\IBM-Datathon\Dataset\raw_data\reddit_data_raw.csv'
df = pd.read_csv(file_path)
df.head()

In [ ]:
device = 0 if torch.cuda.is_available() else -1

if device == 0:
    print('Using GPU')
else:
    print('Using CPU')

pipe = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english", device=device)
pipe.save_pretrained(os.path.join('..', 'src', 'Sentiment', 'sentiment_model'))

In [ ]:
def classify_comment(comment):
    truncated_comment = comment[:512]
    result = pipe(truncated_comment)

    label = result[0]['label']
    return 1 if label == 'POSITIVE' else -1


df['label'] = df['comment'].apply(classify_comment)

In [5]:
folder_path = r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\IBM-Datathon\Dataset\labelled_data'
csv_file_path = os.path.join(folder_path, 'reddit_data_labelled.csv')

df.to_csv(csv_file_path, index=False)

In [6]:
edge_list_path = r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\IBM-Datathon\Dataset\raw_graph\edges_list.csv'
edges_df = pd.read_csv(edge_list_path)

G = nx.from_pandas_edgelist(edges_df, source='source', target='target')